In [ ]:
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from bs4 import BeautifulSoup
from pprint import pprint
import re
import pandas as pd
import io

# указать логин и пароль
# запустить первые три блока, дождаться смс с кодом (указать его в 4м блоке)

username = ''
password = ''

In [ ]:
s = requests.session()

payload = {
"username":username,
"password":password,
"queryRedirectParams":{"response_type":"code",
                       "client_id":"click-web",
                       "scope":"openid click-web",
                       "acr_values":"username",
                       "non_authorized_user":"true"},
"currentRoute":"/password","captchaKey":""}
resp = s.post('https://private.auth.alfabank.ru/passport/cerberus-mini-green/dashboard-green/api/oid/authorize', json=payload, verify=False)
pprint(resp.json())
mfa_token = resp.json()['params']['mfa_token']
username = resp.json()['params']['username']

In [ ]:
data = {
    "queryRedirectParams":{
        "response_type":"code",
        "client_id":"click-web",
        "scope":"openid click-web",
        "acr_values":"username",
        "non_authorized_user":"true"},
    "previousMultiFactorResponseParams":{
        "redirect_uri":"https://click.alfabank.ru/openid/authorize/click-web",
        "mfa_token":mfa_token,
        "acr_values":"username:sms",
        "client_id":"click-web",
        "scope":"openid click-web",
        "role":"client",
        "username":username},
    "is_push":True,
    "type":"CARD"}

resp_ = s.post('https://private.auth.alfabank.ru/passport/cerberus-mini-green/dashboard-green/api/oid/reference', json=data, verify=False)

pprint(resp_.json())
reference = resp_.json()['reference']['reference']
masked_phone = resp_.json()['reference']['masked_phone']

In [ ]:
data = {"credentials":{
            "queryRedirectParams":{
                    "response_type":"code",
                    "client_id":"click-web",
                    "scope":"openid click-web",
                    "acr_values":"username",
                    "non_authorized_user":"true"},
            "previousMultiFactorResponseParams":{
                    "redirect_uri":"https://click.alfabank.ru/openid/authorize/click-web",
                    "mfa_token":mfa_token,
                    "acr_values":"username:sms",
                    "client_id":"click-web",
                    "scope":"openid click-web",
                    "role":"client",
                    "username":username,
                    "reference":reference,
                    "masked_phone":masked_phone},
            "is_push":False,
            "type":"CARD",
            "code":"0000"}} ###############смс-код

resp = s.post('https://private.auth.alfabank.ru/passport/cerberus-mini-green/dashboard-green/api/oid/finishCustomerRegistration', json=data, verify=False)
pprint(resp.json())
code = resp.json()['params']['code']
expires_in = resp.json()['params']['expires_in']
session_token = re.findall('session_token=(.+)&code', resp.json()['redirectUrl'])[0]

In [ ]:
resp = s.get(f"https://click.alfabank.ru/openid/authorize/click-web?redirect_uri=https%3A%2F%2Fclick.alfabank.ru%2Fopenid%2Fauthorize%2Fclick-web&mfa_token={mfa_token}&acr_values=username%3Asms&client_id=click-web&scope=openid+click-web&role=client&username={username}&reference={reference}&masked_phone={masked_phone}&session_token={session_token}&code={code}&expires_in={expires_in}&auth_type=LOGIN", verify=False)
GW_SESSION_CLICK = resp.request.headers['Cookie']

In [ ]:
head = {'Cookie':GW_SESSION_CLICK}
resp = s.get('https://click.alfabank.ru/pfm/hapi/export?dateFrom=2021-03-31T19%3A00%3A00.000Z&dateTo=2021-04-30T18%3A59%3A59.999Z', headers=head, verify=False).content
data=pd.read_csv(io.BytesIO(resp), sep=";", parse_dates=['Дата'], dayfirst=True, index_col='Дата')

In [ ]:
data

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 5)

In [ ]:
data['Сумма'].plot()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Pie(labels=data['Категория'], values=data['Сумма'], hole=.5, textinfo='label+percent')])
fig.update_layout(title=f"Расходы: {sum(data['Сумма'])}")
fig.show()

In [ ]:
# pprint(data.to_dict('records'))
df = data.to_dict('records')
category=['', '']
desc=['Доход', 'Расход']
prof = 0
exp = 0
for i in df:
    if i['Тип']=='Доход':
        prof=prof+i['Сумма']
    else:
        exp=exp+i['Сумма']
values = [prof, exp]
for i in df:
    if i['Категория'] not in desc:
        desc.append(i['Категория'])
        if i['Тип']=='Доход':
            category.append('Доход')
        else:
            category.append('Расход')
        summ = 0
        for j in df:
            if j['Категория']==i['Категория']:
                summ=summ+j['Сумма']
        values.append(summ)
for i in df:
    if i['Описание'] not in desc:
        desc.append(i['Описание'])
        category.append(i['Категория'])
        summ = 0
        for j in df:
            if j['Описание']==i['Описание']:
                summ=summ+j['Сумма']
        values.append(summ)

In [ ]:
import plotly.express as px
data_cat = dict(desc=desc, category=category, values=values)
fig =px.sunburst(
    data_cat,
    names='desc',
    parents='category',
    values='values'
)
fig.show()